In [1]:
import numpy as np
import pandas as pd
import re


# Algorithms
from keras.layers import Flatten,LSTM,Conv1D,Dropout,Bidirectional, GRU, RNN, Concatenate
from keras.utils import plot_model, np_utils
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Input
from keras.models import Model
from keras.layers import Flatten, Conv1D, RNN, LSTM, GlobalAveragePooling1D, Dense
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

topics = train.topic.unique().tolist()

l = list(range(21))
labels = {i : topics[i] for i in range(0, len(topics))}
d = dict(zip(topics, l))
# print(labels)
# print(d)

train['labels'] = train["topic"].apply(lambda x: d[x])
all_data = pd.concat([train, test], ignore_index=False)
titles = []
reviews= []

for i in all_data['Review Title']:
    titles.append(i)

for line in all_data['Review Text']:
    reviews.append(line)

print(len(titles))
print(len(reviews))

8512
8512


/home/akash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [3]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

titles_clean = preprocess_reviews(titles)
reviews_clean = preprocess_reviews(reviews)


X_train = titles_clean[:train.shape[0]]
X_test = titles_clean[train.shape[0]:]
print(len(X_train), len(X_test))
X_trainj = reviews_clean[:train.shape[0]]
X_testj = reviews_clean[train.shape[0]:]
Y = []

for i in train['labels']:
    Y+=[i]

Y = np.asarray(Y)
Y = np_utils.to_categorical(Y, 21)
print(Y.shape)

5959 2553
(5959, 21)


In [4]:
freq_words_state=400

tokenizer_state = Tokenizer(num_words=freq_words_state)
tokenizer_state.fit_on_texts(X_train)


X_train = tokenizer_state.texts_to_sequences(X_train)
# X_val = tokenizer_state.texts_to_sequences(X_val)
X_test = tokenizer_state.texts_to_sequences(X_test)


vocab_size_state = len(tokenizer_state.word_index) + 1
print(vocab_size_state)
maxlen_s = 30

X_train = pad_sequences(X_train, maxlen=maxlen_s, padding='post')
# X_val = pad_sequences(X_val, maxlen=maxlen_s, padding='post')
X_test = pad_sequences(X_test, maxlen=maxlen_s, padding='post')


2547


In [5]:
print(X_train[895])

[ 82 206 114  44  88   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [6]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [8]:
embedding_matrix_s = zeros((vocab_size_state, 50))
for word, index in tokenizer_state.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix_s[index] = embedding_vector
    
print(embedding_matrix_s.shape)

embedding_layer_s = Embedding(vocab_size_state, 50, weights=[embedding_matrix_s],
                              input_length=maxlen_s , trainable=False)
print(embedding_layer_s)

W0810 16:44:34.717592 140455335413568 deprecation_wrapper.py:119] From /home/akash/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



(2547, 50)


In [9]:
freq_words_justify=2000

tokenizer_justify = Tokenizer(num_words=freq_words_justify)
tokenizer_justify.fit_on_texts(X_trainj)

X_trainj = tokenizer_justify.texts_to_sequences(X_trainj)
# X_valj = tokenizer_justify.texts_to_sequences(X_valj)
X_testj = tokenizer_justify.texts_to_sequences(X_testj)
print(len(X_trainj))

5959


In [10]:
vocab_size_justify = len(tokenizer_justify.word_index) + 1
print(vocab_size_justify)
maxlen_j = 150

X_trainj = pad_sequences(X_trainj, maxlen=maxlen_j, padding='post')
# X_valj = pad_sequences(X_valj, maxlen=maxlen_j, padding='post')
X_testj = pad_sequences(X_testj, maxlen=maxlen_j, padding='post')

8763


In [11]:
X_trainj[167]

array([  14,   72,    4,    3,    1,  292,    4,  538,   92,  140,   50,
         34,   27,   76,  192,    5,  219,   23,   11,    5,  514,  161,
        132,  199,    9,   89,   10,    3,  111,   15,  304,  229,   16,
          2, 1062,  406,   34,    9,   43,  499,  406,   29,    5,  775,
         35,   10,  223,  107,   21,   89,  185,    2,   26,    1,  262,
         19,    7,   35,   28,    2,    6,  773,   43, 1470,  183,    8,
        525,   11,    2,  114,  161,   29,    5,  137,   35,   80,   83,
         63,    5, 1063,    8,  183,   66,  143,  209,  311,   11,    2,
        208,  453,  535,   20,  525,   62,  365,  189,   11,   25,  111,
          2,  263,   16,    1,   17,   65,   58,   30,    1,   73, 1348,
         12, 1235,    3,  118,    1,   62,    9,  182,   81,  406,  882,
        189,  311,   19,  274,    4,  538,  140,    3,   59,    5,   51,
        533,   13,    7,   10,    9,    5,  533,   42,   12,  531,    6,
         14,   21,   25,  814,  175,   12,  140], d

In [13]:

embedding_matrix_j = zeros((vocab_size_justify, 50))
for word, index in tokenizer_justify.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix_j[index] = embedding_vector
    
print(embedding_matrix_j.shape)

embedding_layer_j = Embedding(vocab_size_justify, 50, weights=[embedding_matrix_j],
                              input_length=maxlen_j , trainable=False)
print(embedding_layer_j)

(8763, 50)


In [14]:
embedding_matrix_j

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.11891   ,  0.15255   , -0.082073  , ..., -0.57511997,
        -0.26671001,  0.92120999],
       [ 0.41800001,  0.24968   , -0.41242   , ..., -0.18411   ,
        -0.11514   , -0.78580999],
       ...,
       [ 0.080652  , -0.64354002,  0.12345   , ...,  0.13384999,
         0.1115    , -0.56487   ],
       [ 0.28990999, -0.95266002, -0.43441001, ..., -0.41021001,
         1.71749997, -0.39603001],
       [ 0.34303001, -0.18794   ,  0.20774999, ...,  0.23029999,
         0.31827   , -0.40224999]])

In [15]:


seq_input = Input(shape=(maxlen_s, ), dtype='int32')
x = embedding_layer_s(seq_input)
x = Flatten()(x)
x = Dense(21, activation='softmax')(x)

model = Model(inputs=seq_input, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

print(model.summary())

W0810 16:44:49.147285 140455335413568 deprecation_wrapper.py:119] From /home/akash/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 16:44:49.149177 140455335413568 deprecation_wrapper.py:119] From /home/akash/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 16:44:49.154724 140455335413568 deprecation_wrapper.py:119] From /home/akash/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0810 16:44:49.155203 140455335413568 deprecation_wrapper.py:119] From /home/akash/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 50)            127350    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 21)                31521     
Total params: 158,871
Trainable params: 31,521
Non-trainable params: 127,350
_________________________________________________________________
None


In [16]:
# history = model.fit(X_train, Y, batch_size=128, epochs=5, verbose=1)

# loss, acc= model.evaluate(X_train, Y, verbose=1)
# print(acc)
# t = model.predict(X_test)

In [17]:
# print(t.shape)
# print(t[1])
# result = np.where(t[1] == np.amax(t[1]))
# result = np.argmax(t[1])
# print(result)
# result = []
# for i in range(t.shape[0]):
#     result+=[np.argmax(t[i])]

In [18]:
# print(type(result))
# final = []
# for i in result:
#     final.append(labels[i])

In [19]:
seq_input = Input(shape=(maxlen_j, ), dtype='int32')
x = embedding_layer_j(seq_input)
x = Conv1D(128, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(21, activation='softmax')(x)

model = Model(inputs=seq_input, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 50)           438150    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 146, 128)          32128     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 142, 256)          164096    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                5397      
Total params: 639,771
Trainable params: 201,621
Non-trainable params: 438,150
________________________________________________________________

In [22]:
history = model.fit(X_trainj, Y, batch_size=128, epochs=10, verbose=1)

loss, acc= model.evaluate(X_trainj, Y, verbose=1)
print(acc)
t = model.predict(X_testj)

result = []
for i in range(t.shape[0]):
    result+=[np.argmax(t[i])]

Epoch 1/10
5959/5959 [==============================] - 6s 937us/step - loss: 1.3845 - acc: 0.5482
Epoch 2/10
5959/5959 [==============================] - 6s 1ms/step - loss: 1.2924 - acc: 0.5655
Epoch 3/10
5959/5959 [==============================] - 6s 976us/step - loss: 1.1982 - acc: 0.5877
Epoch 4/10
5959/5959 [==============================] - 5s 862us/step - loss: 1.1201 - acc: 0.5971
Epoch 5/10
5959/5959 [==============================] - 5s 801us/step - loss: 1.0608 - acc: 0.6147
Epoch 6/10
5959/5959 [==============================] - 5s 782us/step - loss: 1.0184 - acc: 0.6251
Epoch 7/10
5959/5959 [==============================] - 5s 771us/step - loss: 0.9456 - acc: 0.6283
Epoch 8/10
5959/5959 [==============================] - 5s 825us/step - loss: 0.8934 - acc: 0.6400
Epoch 9/10
5959/5959 [==============================] - 5s 785us/step - loss: 0.8834 - acc: 0.6389
Epoch 10/10
5959/5959 [==============================] - 2s 253us/step
0.6939083739082412


In [21]:
final = []
for i in result:
    final.append(labels[i])

In [23]:
submission = pd.DataFrame({
    'Review Text': test['Review Text'],
    'Review Title':test['Review Title'],
    'topic': final
})

submission.to_csv('submit_dl2.csv', index=False)